This notebook creates the data necessary for both evaluation and selection of waveforms to sort in the current version of DeepSpikeSort (ultimately, it will be used only for evaluation).

In [1]:
from one.api import ONE

one = ONE(
    base_url="https://openalyx.internationalbrainlab.org",
    username="intbrainlab",  
    password="international",
    silent=True
)

eid = "c99d53e6-c317-4c53-99ba-070b26673ac4"
pids, _ = one.eid2pid(eid)
pid = pids[0]

In [ ]:
import os
import spikeinterface.full as si

data_folder = "../data/sub-CSHL049"
extractors_folder = os.path.join(data_folder, "extractors")
sorting_folder = os.path.join(extractors_folder, "sorting")
if not os.path.exists(sorting_folder):  
    sorting = si.read_ibl_sorting(pid)        
    sorting.save(folder=sorting_folder)
else:
    sorting = si.load_extractor(sorting_folder)

sorting

(S3) /global/homes/r/rly/Downloads/ONE/openalyx.internationalbrainlab.org/churchlandlab/Subjects/CSHL049/2020-01-09/001/raw_ephys_data/probe00/_spikeglx_ephysData_g0_t0.imec.ap.ch: 100%|██████████| 129k/129k [00:00<00:00, 285kB/s]
(S3) /global/homes/r/rly/Downloads/ONE/openalyx.internationalbrainlab.org/churchlandlab/Subjects/CSHL049/2020-01-09/001/raw_ephys_data/probe00/_spikeglx_ephysData_g0_t0.imec.ap.meta: 100%|██████████| 16.8k/16.8k [00:00<00:00, 67.7kB/s]
/global/common/software/m3513/deepspikesort/lib/python3.12/site-packages/one/util.py:406: ALFWarning: No default revision for dataset alf/probe00/pykilosort/spikes.amps.npy; using most recent
  warnings.warn(
/global/common/software/m3513/deepspikesort/lib/python3.12/site-packages/one/util.py:406: ALFWarning: No default revision for dataset alf/probe00/pykilosort/spikes.clusters.npy; using most recent
  warnings.warn(
/global/common/software/m3513/deepspikesort/lib/python3.12/site-packages/one/util.py:406: ALFWarning: No defaul

In [ ]:
analyzer_folder = os.path.join(extractors_folder, "analyzer")
if not os.path.exists(analyzer_folder):
    analyzer = si.create_sorting_analyzer(
        sorting=sorting,
        recording=recording_cmr,
        format="memory"
    )
    
    # Compute extensions
    job_kwargs = dict(n_jobs=10, chunk_duration="1s", progress_bar=True)
    compute_dict = {
        'random_spikes': {'method': 'uniform'},
        'waveforms': {'ms_before': 1.0, 'ms_after': 2.0},
        'templates': {'operators': ["average", "median", "std"]}
    }
    analyzer.compute(compute_dict, **job_kwargs)
    
    # Save the sorting analyzer to disk
    analyzer.save_as(folder=analyzer_folder, format="binary_folder")
else:
    analyzer = si.load_sorting_analyzer(analyzer_folder)
    
analyzer

In [ ]:
channel_locations_file = os.path.join(extractors_folder, "channel_locations.npy")
channel_locations= np.load(channel_locations_file)

In [ ]:
spikes_folder = os.path.join(data_folder, "spikes")
os.makedirs(spikes_folder, exist_ok=True)

spikes_file = os.path.join(spikes_folder, "spikes.npy")
if not os.path.exists(spikes_file):
    spikes = preprocessing.extract_spikes(sorting, analyzer, channel_locations)
    np.save(spikes_file, spikes)
else:
    spikes = np.load(spikes_file)
    
display(pd.DataFrame(spikes))